In [1]:
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmcv.parallel import MMDataParallel
from mmdet3d.datasets import build_dataset
from mmdet3d.models import build_model
from mmcv.runner import get_dist_info, init_dist, load_checkpoint, wrap_fp16_model
from mmcv import Config, DictAction
import warnings
from timeit import default_timer as timer
import torch.utils.benchmark as benchmark
import torch.nn as nn
import torch
import os
import time
import logging
from custome_logger import setup_custom_logger
logger = setup_custom_logger()
logger.debug("test")


In [5]:
def import_modules_load_config(cfg_file="beverse_tiny.py", samples_per_gpu=1):
    cfg_path = r"/home/niklas/ETM_BEV/BEVerse/projects/configs"
    cfg_path = os.path.join(cfg_path, cfg_file)

    cfg = Config.fromfile(cfg_path)

    # if args.cfg_options is not None:
    #     cfg.merge_from_dict(args.cfg_options)
    # import modules from string list.
    if cfg.get("custom_imports", None):
        from mmcv.utils import import_modules_from_strings

        import_modules_from_strings(**cfg["custom_imports"])

    # import modules from plguin/xx, registry will be updated
    if hasattr(cfg, "plugin"):
        if cfg.plugin:
            import importlib

            if hasattr(cfg, "plugin_dir"):
                plugin_dir = cfg.plugin_dir
                _module_dir = os.path.dirname(plugin_dir)
                _module_dir = _module_dir.split("/")
                _module_path = _module_dir[0]

                for m in _module_dir[1:]:
                    _module_path = _module_path + "." + m
                print(_module_path)
                plg_lib = importlib.import_module(_module_path)
            else:
                # import dir is the dirpath for the config file
                _module_dir = cfg_path
                _module_dir = _module_dir.split("/")
                _module_path = _module_dir[0]
                for m in _module_dir[1:]:
                    _module_path = _module_path + "." + m
                print(_module_path)
                plg_lib = importlib.import_module(_module_path)

    samples_per_gpu = 1
    if isinstance(cfg.data.test, dict):
        cfg.data.test.test_mode = True
        samples_per_gpu = cfg.data.test.pop("samples_per_gpu", 1)
        if samples_per_gpu > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            cfg.data.test.pipeline = replace_ImageToTensor(
                cfg.data.test.pipeline)
    elif isinstance(cfg.data.test, list):
        for ds_cfg in cfg.data.test:
            ds_cfg.test_mode = True
        samples_per_gpu = max(
            [ds_cfg.pop("samples_per_gpu", 1) for ds_cfg in cfg.data.test]
        )
        if samples_per_gpu > 1:
            for ds_cfg in cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    return cfg


torch.backends.cudnn.benchmark = True


In [6]:
cfg = import_modules_load_config(
    cfg_file=r"petr/petr_r50dcn_gridmask_p4.py")

cfg.data_root = '/home/niklas/ETM_BEV/BEVerse/data/nuscenes/'


projects.mmdet3d_plugin


In [11]:
cfg.data_root


'/home/niklas/ETM_BEV/BEVerse/data/nuscenes/'

In [5]:
cfg.keys()

dict_keys(['point_cloud_range', 'class_names', 'dataset_type', 'data_root', 'input_modality', 'file_client_args', 'train_pipeline', 'test_pipeline', 'eval_pipeline', 'data', 'evaluation', 'checkpoint_config', 'log_config', 'dist_params', 'log_level', 'work_dir', 'load_from', 'resume_from', 'workflow', 'backbone_norm_cfg', 'plugin', 'plugin_dir', 'voxel_size', 'img_norm_cfg', 'model', 'db_sampler', 'ida_aug_conf', 'optimizer', 'optimizer_config', 'lr_config', 'total_epochs', 'find_unused_parameters', 'runner'])

In [4]:
cfg["data"]["test"]["data_root"] = '/home/niklas/ETM_BEV/BEVerse/data/nuscenes/'
cfg["data"]["test"]["ann_file"] = '/home/niklas/ETM_BEV/BEVerse/data/nuscenes/nuscenes_infos_val.pkl'


In [7]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=2,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

sample = next(iter(data_loader))


In [12]:
gt_bboxes_list= (sample["gt_bboxes_3d"].data[0])


In [66]:
gt_bboxes_list = [torch.cat(
    (gt_bboxes.gravity_center, gt_bboxes.tensor[:, 3:]),
    dim=1).to("cpu") for gt_bboxes in gt_bboxes_list]


In [67]:
for gt_list in gt_bboxes_list:
    print(gt_list.shape)


torch.Size([50, 9])
torch.Size([51, 9])
torch.Size([56, 9])
torch.Size([61, 9])


In [55]:
gt_bboxes_list[0][0]


tensor([ 2.8615e+01,  2.7538e+01, -2.4884e-02,  7.3753e-01,  7.3182e-01,
         1.6283e+00, -2.8156e+00,  3.6043e-01,  1.1425e+00])

In [26]:
sample.keys()

dict_keys(['img_metas', 'gt_bboxes_3d', 'gt_labels_3d', 'img'])

In [8]:
model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))


model.cuda()
model = MMDataParallel(model, device_ids=[0])


PETR HEAD
query_embedding
 Sequential(
  (0): Linear(in_features=384, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
) 

reference_points
 Embedding(900, 3) 

reg_branches
 ModuleList(
  (0): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
  (1): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
  (2): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
  (3): Sequential(
    (0): Linear(in_f

In [6]:
sample.keys()

dict_keys(['img_metas', 'img'])

In [ ]:
sample['img_metas']


[DataContainer([[{'filename': ['./data/nuscenes/samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151603512404.jpg', './data/nuscenes/samples/CAM_FRONT_RIGHT/n008-2018-08-01-15-16-36-0400__CAM_FRONT_RIGHT__1533151603520482.jpg', './data/nuscenes/samples/CAM_FRONT_LEFT/n008-2018-08-01-15-16-36-0400__CAM_FRONT_LEFT__1533151603504799.jpg', './data/nuscenes/samples/CAM_BACK/n008-2018-08-01-15-16-36-0400__CAM_BACK__1533151603537558.jpg', './data/nuscenes/samples/CAM_BACK_LEFT/n008-2018-08-01-15-16-36-0400__CAM_BACK_LEFT__1533151603547405.jpg', './data/nuscenes/samples/CAM_BACK_RIGHT/n008-2018-08-01-15-16-36-0400__CAM_BACK_RIGHT__1533151603528113.jpg'], 'ori_shape': (900, 1600, 3, 6), 'img_shape': [(512, 1408, 3), (512, 1408, 3), (512, 1408, 3), (512, 1408, 3), (512, 1408, 3), (512, 1408, 3)], 'lidar2img': [array([[ 1.09383099e+03,  7.39771576e+02,  2.88310468e+01,
         -3.11829001e+02],
        [-1.27404083e+01,  1.92810809e+02, -1.09359924e+03,
         -4.48285911e+02],


In [9]:
with torch.no_grad():
    result = model(
        return_loss=False,
        rescale=True,
        img_metas=sample['img_metas'],
        img=sample['img'],
        #future_egomotions=sample['future_egomotions'],
        #motion_targets=motion_distribution_targets,
        #img_is_valid=sample['img_is_valid'][0],
    )


PETR_HEAD FORWARD Input: x.shape =torch.Size([2, 6, 256, 32, 88])
PETR_HEAD FORWARD Input Proj: x.shape =torch.Size([12, 256, 32, 88])
PETR_HEAD FORWARD Mask: torch.Size([2, 6, 32, 88])
PETR_HEAD FORWARD posembed: torch.Size([2, 6, 256, 32, 88])
POS SINE: pos_n.shape =torch.Size([2, 6, 32, 88, 128]) pos_x.shape =torch.Size([2, 6, 32, 88, 128]) pos_y.shape =torch.Size([2, 6, 32, 88, 128]), pos.shape =torch.Size([2, 6, 384, 32, 88])  mask.shape =torch.Size([2, 6, 32, 88])
PETR_HEAD FORWARD reference_points: torch.Size([900, 3])
pos2posemb3d input:  torch.Size([900, 3])
pos_x.shape = torch.Size([900, 128]), pos_y.shape = torch.Size([900, 128]), pos_z.shape = torch.Size([900, 128]), posemb.shape = torch.Size([900, 384])
PETR_HEAD FORWARD query_embeds: torch.Size([900, 256])
PETR_HEAD FORWARD reference_points: torch.Size([2, 900, 3])
memory.shape = torch.Size([16896, 2, 256])
pos_embed.shape = torch.Size([16896, 2, 256])
query_embed.shape = torch.Size([900, 2, 256])
mask.shape = torch.Size(

In [22]:
(result[0]["pts_bbox"]["boxes_3d"].tensor.shape)  # labels_3d


torch.Size([300, 9])

What do I 